# This notebook explores the most similar documents with KNN and one shot SVM, to see how good is the

# Aprender a programar Basic set up

#### install libraries

In [1]:
#from google.colab import files
#upload = files.upload()
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [2]:
#models sentence bert
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 730.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9909ab53b8504635a7fa548efa57a997f83a58ab288f3eb3275b4d632317070d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
#
import re
import numpy as np
import pandas as pd
from pprint import pprint

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing  # to normalize data for cosine similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
#from gensim.models import CoherenceModel

# spacy for lemmatization
!pip install -U spacy
!python -m spacy download es
!python -m spacy download es_core_news_md
import spacy

# Plotting tools
#!pip install pyLDAvis
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline


#knee locator for clustering
!pip install kneed
!pip install ipython
from kneed import KneeLocator

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)


# umap dimensionality reduction for the 2d graphs
!pip install umap-learn

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

2023-09-06 22:04:35.020903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 35.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
2023-09-06 22:05:14.433577: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 21.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 961.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.

In [4]:
#!pip install tensorflow==2.11.0
import tensorflow as tf
print(tf.__version__)

2.12.0


In [5]:
#tf_version = str(tf.__version__)[:-2]
#print(tf_version)
!pip install --quiet tensorflow-text==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 39.9 MB/s eta 0:00:00


In [6]:
import string
import seaborn as sns
nlp = spacy.load('es_core_news_md')

### data visualization

In [7]:
df = pd.read_csv(r'/content/mydrive/MyDrive/luis/study/master/tesis/automaticLabels/aprenderAProgramarFinalDataFrameWithCalificacion.csv',sep=';')
df

,comment,ambiente,bondadesPlataforma,CbondadesPlataformaCcasosPrueba,CbondadesPlataformaCconocerErrores,CbondadesPlataformaCconsejosFormativos,CbondadesPlataformaCdisponibilidadLinea,CbondadesPlataformaCespacioTrabajo,CbondadesPlataformaCfacilidadUso,CbondadesPlataformaClenguajes,...,CMejorableAprenderProgramarCfallasGenerales,CMejorableAprenderProgramarCinflexibilidad,CMejorableAprenderProgramarCobjetivoEducativo,CMejorableAprenderProgramarCretroalimentacionInsuficiente,CMejorableAprenderProgramarCsustituible,colab,HerramientasUNcode,logrosPedagogicos,MejorableAprenderProgramar,calificacion
0,A traves de la plataforma pude ir practicando ...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NO CALIFICADO
1,A traves de los ejercicios propuestos en la he...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Totalmente de acuerdo
2,A veces UNCode no dejaba correr por cosas triv...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,Algo de acuerdo
3,A veces es poco practico y da errores por cosa...,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,Algo en desacuerdo
4,"A veces la pagina era muy lenta, y al principi...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,NO CALIFICADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,si porque gracias a el pude constatar que mis ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,De acuerdo
343,"si, ya que es bastante sencillo e intuitivo a ...",0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,NO CALIFICADO
344,solo lo use para enviar los talleres ya que lo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,En desacuerdo
345,uno de los aspectos que considero que mas me c...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Totalmente de acuerdo


In [8]:
df.comment

0      A traves de la plataforma pude ir practicando ...
1      A traves de los ejercicios propuestos en la he...
2      A veces UNCode no dejaba correr por cosas triv...
3      A veces es poco practico y da errores por cosa...
4      A veces la pagina era muy lenta, y al principi...
                             ...                        
342    si porque gracias a el pude constatar que mis ...
343    si, ya que es bastante sencillo e intuitivo a ...
344    solo lo use para enviar los talleres ya que lo...
345    uno de los aspectos que considero que mas me c...
346    ya que proporcionaba todas las herramientas ne...
Name: comment, Length: 347, dtype: object

This shows there are comments with multiple categories, on average each comment will have 1.5 categories

In [9]:
df.shape

(347, 30)

### data selection

In [10]:
df[df.comment.str.len() > 5].shape

(347, 30)

In [11]:
df['calificacion'].unique()

array(['NO CALIFICADO', 'Totalmente de acuerdo', 'Algo de acuerdo',
       'Algo en desacuerdo', 'De acuerdo', 'Totalmente en desacuerdo',
       'En desacuerdo'], dtype=object)

In [12]:
# items that have values different than one and zero(they appear nmore than once in the dataset of catefgories)
# for i in range(len(df.bondadesPlataforma)):
#   if (df.bondadesPlataforma[i] ==2):
#     print(df.comment[i])

In [13]:
# some comments appear two times in the same categorie, in the text data, so we replace the number 2 for a 1
df = df.replace(2, 1)

In [14]:
# Drop the column you want to omit
df_test = df.drop('comment', axis=1)

# check all values in all columns are what we expect
for col in df_test .columns:
  print(df_test[col].unique())

[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
['NO CALIFICADO' 'Totalmente de acuerdo' 'Algo de acuerdo'
 'Algo en desacuerdo' 'De acuerdo' 'Totalmente en desacuerdo'
 'En desacuerdo']


In [15]:
# final working dataframe
df.reset_index(drop=True, inplace=True)
df

,comment,ambiente,bondadesPlataforma,CbondadesPlataformaCcasosPrueba,CbondadesPlataformaCconocerErrores,CbondadesPlataformaCconsejosFormativos,CbondadesPlataformaCdisponibilidadLinea,CbondadesPlataformaCespacioTrabajo,CbondadesPlataformaCfacilidadUso,CbondadesPlataformaClenguajes,...,CMejorableAprenderProgramarCfallasGenerales,CMejorableAprenderProgramarCinflexibilidad,CMejorableAprenderProgramarCobjetivoEducativo,CMejorableAprenderProgramarCretroalimentacionInsuficiente,CMejorableAprenderProgramarCsustituible,colab,HerramientasUNcode,logrosPedagogicos,MejorableAprenderProgramar,calificacion
0,A traves de la plataforma pude ir practicando ...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NO CALIFICADO
1,A traves de los ejercicios propuestos en la he...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Totalmente de acuerdo
2,A veces UNCode no dejaba correr por cosas triv...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,Algo de acuerdo
3,A veces es poco practico y da errores por cosa...,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,Algo en desacuerdo
4,"A veces la pagina era muy lenta, y al principi...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,NO CALIFICADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,si porque gracias a el pude constatar que mis ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,De acuerdo
343,"si, ya que es bastante sencillo e intuitivo a ...",0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,NO CALIFICADO
344,solo lo use para enviar los talleres ya que lo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,En desacuerdo
345,uno de los aspectos que considero que mas me c...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Totalmente de acuerdo


#### copy of df with calification

In [16]:
# convert calification value into a sparse numerical value to treat it as a categorical variable in the future models
# Create dummy variables for the 'calificacion' column
calificacion_dummies = pd.get_dummies(df['calificacion'], prefix='calificacion')

temp_df = pd.concat([df, calificacion_dummies], axis=1)

# convert to dense dataframe, select only cALIFICACION
temp_calification_statistics= [col for col in temp_df.columns if col.startswith('calificacion') and not col.startswith('calificacion_encoded')]
temp_calification_statistics_labels = temp_df[temp_calification_statistics]
temp_calification_statistics_labels.sum()

calificacion                             NO CALIFICADOTotalmente de acuerdoAlgo de acue...
calificacion_Algo de acuerdo                                                            30
calificacion_Algo en desacuerdo                                                         12
calificacion_De acuerdo                                                                108
calificacion_En desacuerdo                                                               4
calificacion_NO CALIFICADO                                                              84
calificacion_Totalmente de acuerdo                                                     105
calificacion_Totalmente en desacuerdo                                                    4
dtype: object

In [17]:
# Create a dictionary that maps the text values to numerical values
calificacion_mapping = {
    'Totalmente de acuerdo': 3,
 'De acuerdo': 2,
   'Algo de acuerdo': 1,
      'Algo en desacuerdo': 0,
   'En desacuerdo': -1,
  'Totalmente en desacuerdo': -2
}

df_calification = pd.DataFrame()

temp_df.drop('calificacion', axis=1, inplace=True)

# Apply the mapping to the 'calificacion' column
df_calification['calificacion_encoded'] = df['calificacion'].map(calificacion_mapping)
df_calification = pd.concat([df_calification, calificacion_dummies], axis=1)
df_calification['comment'] = df['comment']
df_calification

,calificacion_encoded,calificacion_Algo de acuerdo,calificacion_Algo en desacuerdo,calificacion_De acuerdo,calificacion_En desacuerdo,calificacion_NO CALIFICADO,calificacion_Totalmente de acuerdo,calificacion_Totalmente en desacuerdo,comment
0,NaN,0,0,0,0,1,0,0,A traves de la plataforma pude ir practicando ...
1,3.0,0,0,0,0,0,1,0,A traves de los ejercicios propuestos en la he...
2,1.0,1,0,0,0,0,0,0,A veces UNCode no dejaba correr por cosas triv...
3,0.0,0,1,0,0,0,0,0,A veces es poco practico y da errores por cosa...
4,NaN,0,0,0,0,1,0,0,"A veces la pagina era muy lenta, y al principi..."
...,...,...,...,...,...,...,...,...,...
342,2.0,0,0,1,0,0,0,0,si porque gracias a el pude constatar que mis ...
343,NaN,0,0,0,0,1,0,0,"si, ya que es bastante sencillo e intuitivo a ..."
344,-1.0,0,0,0,1,0,0,0,solo lo use para enviar los talleres ya que lo...
345,3.0,0,0,0,0,0,1,0,uno de los aspectos que considero que mas me c...


#### basic text cleaning

In [18]:
df["comment"] = df["comment"].fillna("")
# for col in text_columns: # change df tyoes to string, it it is not
#     df[col] = df[col].astype(str)

In [19]:
#limpieza basica que aplica para todo metodo sin importar cual sera
#method that overrides the given dataframe text, with a cleaned version
def clean(textArray):
  res= []
  for i in range(0,len(textArray)):
    text = textArray[i]
    #text = p.clean(text)
    #text = re.sub(r'\W+', ' ', text)  # remove non-alphanumeric characters
    text = re.sub(r"\d+", "number", text)# replace numbers with the word 'number'
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    # text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    # text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    # text = re.sub(
    #     f"[{re.escape(string.punctuation)}]", "", text
    # )  # Remove punctuation
    # don't consider sentenced with less than 3 words (i.e. assumed noise)
    #if len(text.strip().split()) < 3:
    #     return None
    text = text.lower()  # lower case everything
    test = text.strip() # remove redundant spaces
    textArray[i] = text
  return

In [20]:
clean(df.comment) # df.comment will be cleaned and overwriten

<ipython-input-19-ee90dfb124bd>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textArray[i] = text


In [21]:
df.head(3)

,comment,ambiente,bondadesPlataforma,CbondadesPlataformaCcasosPrueba,CbondadesPlataformaCconocerErrores,CbondadesPlataformaCconsejosFormativos,CbondadesPlataformaCdisponibilidadLinea,CbondadesPlataformaCespacioTrabajo,CbondadesPlataformaCfacilidadUso,CbondadesPlataformaClenguajes,...,CMejorableAprenderProgramarCfallasGenerales,CMejorableAprenderProgramarCinflexibilidad,CMejorableAprenderProgramarCobjetivoEducativo,CMejorableAprenderProgramarCretroalimentacionInsuficiente,CMejorableAprenderProgramarCsustituible,colab,HerramientasUNcode,logrosPedagogicos,MejorableAprenderProgramar,calificacion
0,a traves de la plataforma pude ir practicando ...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NO CALIFICADO
1,a traves de los ejercicios propuestos en la he...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Totalmente de acuerdo
2,a veces uncode no dejaba correr por cosas triv...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,Algo de acuerdo


### additional data transformation step to convert similar words so results are more reliable / data train/test splitting

In [22]:
#limpieza basica que aplica para todo metodo sin importar cual sera
#method that overrides the given dataframe text, with a cleaned version
def advanceCleaning(textArray):
  res= []
  for i in range(0,len(textArray)):
    text = textArray[i]
    #text = p.clean(text)
    #text = re.sub(r"uncode|plataforma|herramienta", "uncode", text)  # convert common words to the same synonym
    textArray[i] = text
  return

In [ ]:
#comment this cell
#advanceCleaning(df.comment) # df.comment will be cleaned and overwriten

#### Dataset splitting in categories and test splitting

In [23]:
testSize = 0.01
random_state = 50

In [24]:
# convert to dense dataframe, select only the columns of the big categories
selected_columns_only_categories = [col for col in df.columns if not col.startswith('C') and not col.startswith('calificacion')]
selected_columns_only_categories.remove('comment')
categories_labels = df[selected_columns_only_categories]
categories_labels.sum()

ambiente                       14
bondadesPlataforma            167
colab                          10
HerramientasUNcode            128
logrosPedagogicos             187
MejorableAprenderProgramar     86
dtype: int64

In [25]:
# convert to dense dataframe, select only the columns of the small subcategories
selected_columns_only_sub_categories = [col for col in df.columns if col.startswith('C')]
sub_categories_labels = df[selected_columns_only_sub_categories]
sub_categories_labels.sum()

CbondadesPlataformaCcasosPrueba                              10
CbondadesPlataformaCconocerErrores                           82
CbondadesPlataformaCconsejosFormativos                       30
CbondadesPlataformaCdisponibilidadLinea                       9
CbondadesPlataformaCespacioTrabajo                            4
CbondadesPlataformaCfacilidadUso                             43
CbondadesPlataformaClenguajes                                 4
CbondadesPlataformaCPracticaconstante                        20
CherramientasUNcodeCcustomInput                              32
CherramientasUNcodeClinter                                   28
CherramientasUNcodeCpythonTutor                              75
ClogrosPedagogicosCaprendizajeAutonomo                       13
ClogrosPedagogicosCejerciciosEstimulantes                    39
ClogrosPedagogicosCevalluacionEstimulante                    39
ClogrosPedagogicosCEvaluacionOptimizada                      78
ClogrosPedagogicosChabilidadesProgramaci

In [26]:
X_train, X_test, y_train_categories, y_test_categories = train_test_split( df.comment.to_numpy(), categories_labels.to_numpy(), test_size=testSize, random_state=random_state)

In [27]:
X_train, X_test, y_train_sub_categories, y_test_sub_categories = train_test_split( df.comment.to_numpy(), sub_categories_labels.to_numpy(), test_size=testSize, random_state=random_state)

In [28]:
y_train_sub_categories

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

##### split calification into the same trsining and test

In [29]:
# convert to dense dataframe, select only the columns of the small subcategories
selected_columns_only_calification_categories = [col for col in df_calification.columns if not col.startswith('calificacion_encoded') and not col.startswith('comment') ]
calification_categories_labels = df_calification[selected_columns_only_calification_categories]
calification_categories_labels.sum()

calificacion_Algo de acuerdo              30
calificacion_Algo en desacuerdo           12
calificacion_De acuerdo                  108
calificacion_En desacuerdo                 4
calificacion_NO CALIFICADO                84
calificacion_Totalmente de acuerdo       105
calificacion_Totalmente en desacuerdo      4
dtype: int64

In [30]:
X_train_calification, X_test_calification, train_calification_categorical, test_calification_categorical = train_test_split( df_calification.comment.to_numpy(), calification_categories_labels.to_numpy(), test_size=testSize, random_state=random_state)

In [31]:
X_train_calification

array(['Es un buen programa, lo unico es que es algo complicado de usarlo, puesto que las salidas de los codigos, deben seguir una estructura y orden especificos.',
       'No lo usamos muchas veces, y al usarlo, nos obligaba a mejorar el programa',
       'Permitio la creacion de  varios programas en un sistema facil y accesible sin necesidad de descargar el programa de python',
       'Es una buena forma de calificar, aunque hay que ser demasiado especifico en los resultados',
       'Es muy util el hecho de que pruebe casos y que ayude a probar los programas con varios ejemplos que puedan mostrar fallos',
       'Porque permite aplicar los conocimientos ensenados sobre programacion de forma similar a la que se haria en un computador por medio de python.',
       'Porque es online y es facil de usar, ademas gracias al juez virtual y los casos de prueba es mas facil entender como funciona cada ejercicio.',
       'Uncode fue una buena herramienta para programar, pero presenta algunos 

In [32]:
X_train_calification, X_test_calification, train_calification_encoded, test_calification_encoded = train_test_split( df_calification.comment.to_numpy(), df_calification.calificacion_encoded.to_numpy(), test_size=testSize, random_state=random_state)

In [33]:
train_calification_encoded = np.reshape(train_calification_encoded, (-1, 1))
test_calification_encoded = np.reshape(test_calification_encoded, (-1, 1))

In [34]:
train_calification_encoded

array([[ 1.],
       [ 1.],
       [nan],
       [nan],
       [ 3.],
       [ 2.],
       [nan],
       [ 1.],
       [nan],
       [ 2.],
       [ 2.],
       [ 2.],
       [ 2.],
       [ 0.],
       [ 3.],
       [ 3.],
       [-1.],
       [ 2.],
       [ 3.],
       [ 2.],
       [ 2.],
       [ 3.],
       [ 3.],
       [ 3.],
       [ 1.],
       [-2.],
       [nan],
       [ 2.],
       [ 3.],
       [ 0.],
       [ 3.],
       [ 1.],
       [ 2.],
       [nan],
       [nan],
       [nan],
       [-1.],
       [ 3.],
       [ 3.],
       [ 2.],
       [-2.],
       [ 2.],
       [ 2.],
       [ 3.],
       [ 3.],
       [ 2.],
       [ 3.],
       [ 3.],
       [ 3.],
       [nan],
       [ 2.],
       [ 2.],
       [ 1.],
       [ 3.],
       [ 3.],
       [ 2.],
       [ 3.],
       [ 3.],
       [ 2.],
       [nan],
       [ 2.],
       [ 3.],
       [ 3.],
       [ 2.],
       [ 2.],
       [ 3.],
       [ 2.],
       [nan],
       [ 2.],
       [ 2.],
       [ 2.],
      

#### resulting arrays

X_train, X_test = are the cleaned comments texts to perform.

test_calification_encoded and train_calification_encoded contain the one hot encoded rating given by the student (possible aditional input feature)

train_calification_categorical, test_calification_categorical have the rating given by student as categorical data(6 different columns)

y_train_categories, y_test_categories    are the categories corresponding to each comment

y_train_sub_categories, y_test_sub_categories  are the subxategories corersponding to each value

In [35]:
global_data = {
    'all_corpus':{
       'comments': df.comment.to_numpy(),
       'categories': categories_labels.to_numpy(),
       'subcategories':  sub_categories_labels.to_numpy()
    },
    'x': {
        'X_train': X_train,
        'X_test': X_test,
        'test_calification_encoded':test_calification_encoded,
        'train_calification_encoded': train_calification_encoded,
        'train_calification_categorical': train_calification_categorical,
        'test_calification_categorical':test_calification_categorical
    },
    'y' :{
        'y_train_categories':y_train_categories,
       'y_test_categories':y_test_categories ,
       'y_train_sub_categories':y_train_sub_categories,
       'y_test_sub_categories':y_test_sub_categories ,
       },
    'embeddings':{
        "mini_lm": {},
        "use": {},
        "tfid":{},
        'glove':{},
        "all-MiniLM-L6-v2":{},
        "all-MiniLM-L12-v2":{},
        "paraphrase-multilingual-mpnet-base-v2":{},
        "paraphrase-MiniLM-L3-v2":{},
        "concatenation_mini_l_384_dimmentions_and_tfid_stemmized":{},
         "concatenation_mini_lm_with_categorical_calification_dimmentions_and_tfid_stemmized":{},
        "concatenation_mini_lm_with_one_hot_encoded_calification_dimmentions_and_tfid_stemmized":{}
    }
}

In [36]:
model_data = {
    'svm': {
    },
    'gaussian' :{
       },
    'multinomial_bayes':{
    },
    "nn":{},
    'random_forest':{}
}

In [37]:
global_data['x']

{'X_train': array(['es un buen programa, lo unico es que es algo complicado de usarlo, puesto que las salidas de los codigos, deben seguir una estructura y orden especificos.',
        'no lo usamos muchas veces, y al usarlo, nos obligaba a mejorar el programa',
        'permitio la creacion de varios programas en un sistema facil y accesible sin necesidad de descargar el programa de python',
        'es una buena forma de calificar, aunque hay que ser demasiado especifico en los resultados',
        'es muy util el hecho de que pruebe casos y que ayude a probar los programas con varios ejemplos que puedan mostrar fallos',
        'porque permite aplicar los conocimientos ensenados sobre programacion de forma similar a la que se haria en un computador por medio de python.',
        'porque es online y es facil de usar, ademas gracias al juez virtual y los casos de prueba es mas facil entender como funciona cada ejercicio.',
        'uncode fue una buena herramienta para programar, pero

### Aditional optional text cleaning/normalization  stemming and lemmatization

In [38]:
#normalizacion especifica, dependiendo del metodo se puede probar con y sin esta
def normalize(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if t.isalpha()] #len(t) > 3 and
    return lexical_tokens

#otra libreria para eliminar stop words
# import nltk
# from stop_words import get_stop_words
# dataset = ['This','is','just', 'a', 'snake']
# A = [word for word in dataset if word not in get_stop_words('english')]
# print(A)

In [39]:
textNormalizedCommentsVector = [ ([str(a) for a in normalize(oneComment)]) for oneComment in global_data['x']['X_train']]
textNormalizedCommentsVector[0]

['programa',
 'unico',
 'complicado',
 'usarlo',
 'puesto',
 'salidas',
 'codigos',
 'seguir',
 'estructura',
 'orden',
 'especificos']

In [40]:
textNormalizedComments = [ " ".join([str(a) for a in normalize(oneComment)]) for oneComment in global_data['x']['X_train']]
textNormalizedComments[0]

'programa unico complicado usarlo puesto salidas codigos seguir estructura orden especificos'

In [41]:
textNormalizedCommentsTest = [ " ".join([str(a) for a in normalize(oneComment)]) for oneComment in global_data['x']['X_test']]

textNormalizedAllComments= [ " ".join([str(a) for a in normalize(oneComment)]) for oneComment in global_data['all_corpus']['comments']]

textNormalizedCommentsTest[0]

'plataforma problemas util herramientas facilidad usarlo distintas actividades propuestas clase'

In [42]:
#comment this cell
global_data['x']['X_train_normalized']  = textNormalizedComments
global_data['x']['X_test_normalized'] = textNormalizedCommentsTest
global_data['all_corpus']['comments_normalized'] = textNormalizedAllComments

#### stemming optional process to transfrom all word to the root(even if the root does not exist)

In [43]:
import nltk
from nltk import SnowballStemmer

def stemmize(text):
  spanishstemmer=SnowballStemmer('spanish')
  tokens = normalize(text) # crear una lista de tokens
  stems = [spanishstemmer.stem(token) for token in tokens]
  return stems

In [44]:
stemmize(textNormalizedComments[0])

['program',
 'unic',
 'complic',
 'usarl',
 'puest',
 'sal',
 'codig',
 'segu',
 'estructur',
 'orden',
 'especif']

In [45]:
global_data['x']['X_train_stemmized'] = [ " ".join([str(a) for a in stemmize(oneComment)]) for oneComment in global_data['x']['X_train_normalized']]
global_data['x']['X_test_stemmized'] = [ " ".join([str(a) for a in stemmize(oneComment)]) for oneComment in global_data['x']['X_test_normalized']]
global_data['all_corpus']['comments_stemmized'] = [ " ".join([str(a) for a in stemmize(oneComment)]) for oneComment in global_data['all_corpus']['comments_normalized']]
global_data['x']['X_train_stemmized']

['program unic complic usarl puest sal codig segu estructur orden especif',
 'usarl oblig mejor program',
 'permiti creacion program sistem facil acces neces descarg program python',
 'form calific especif result',
 'util prueb cas ayud prob program ejempl pued mostr fall',
 'permit aplic conoc ensen program form simil hari comput python',
 'onlin facil graci juez virtual cas prueb facil entend funcion ejercici',
 'uncod herramient program present factor deberi mejor pagin suel caers ocasion hac optim',
 'moment servidor encuentr dispon uncod herramient util aprendizaj program',
 'plataform corr program vist clas rar error plataform congel pagin razon habl rest plataform excelent resalt herramient permit suced codig line line',
 'util verific fall segu codig pas pas',
 'herramient comprob program pid',
 'herramient sencill util facil entend',
 'cort camp acept program evit cre multipl opcion solucion ejercici',
 'capaz mostr coincident sal respuest esper permit visualiz cambi variabl m

#### lemmatization, optional procces to transform words to a common one

In [46]:
def lemmanize(text):
    doc = nlp(text)
    #lemmas_no_pron = [tok.lemma_.lower() for tok in doc if tok.pos_ != 'PRON'] # we can select the type of tokens that will enter this process, like pronouns, adjetivo, verbo, sustantivo
    #lemmas = [tok.lemma_ for tok in doc if tok.pos_ != 'NOUN']
    lemmas = []
    for tok in doc:
      if tok.pos_ not in ("NOUN", "NUM", "PROPN"):
        lemmas.append((tok.lemma_))
      else:
        lemmas.append((tok.text))
    return lemmas


In [47]:
lemmanize(textNormalizedComments[0])

['programa',
 'unico',
 'complicado',
 'usar él',
 'puesto',
 'salidas',
 'codigos',
 'seguir',
 'estructura',
 'orden',
 'especifico']

In [48]:
dataLemmatizedCommentsVector = [ ([str(a) for a in lemmanize(oneComment)]) for oneComment in textNormalizedComments]
dataLemmatizedCommentsVector[0]

['programa',
 'unico',
 'complicado',
 'usar él',
 'puesto',
 'salidas',
 'codigos',
 'seguir',
 'estructura',
 'orden',
 'especifico']

In [49]:
global_data['x']['X_train_lemmanized'] = [ " ".join([str(a) for a in lemmanize(oneComment)]) for oneComment in global_data['x']['X_train_normalized']]
global_data['x']['X_test_lemmanized'] = [ " ".join([str(a) for a in lemmanize(oneComment)]) for oneComment in global_data['x']['X_test_normalized']]

global_data['all_corpus']['comments_lemmanized'] = [ " ".join([str(a) for a in lemmanize(oneComment)]) for oneComment in global_data['all_corpus']['comments']]

global_data['x']['X_train_lemmanized']

['programa unico complicado usar él puesto salidas codigos seguir estructura orden especifico',
 'usar él obligar mejorar programa',
 'permitio creacion programas sistema facil accesible necesidad descargar programa python',
 'forma calificar especifico resultados',
 'util pruebir casos ayudar probar programas ejemplos poder mostrar fallos',
 'permitir aplicar conocimientos ensenado programacion forma similar hario computador python',
 'online facil gracias juez virtual casos probar facil entender funcionar ejercicio',
 'uncodir herramienta programar presentar factores deberiar mejorar él pagina soler caer él ocasiones hacer él optimo',
 'momentos servidor encontrar disponible uncodir herramienta util aprendizaje programacion',
 'plataforma correr programas visto clase raro errores plataforma congelar pagina razon hablar resto plataforma excelente resalto herramienta permitir suceder codigo linea linea',
 'util verificar fallos seguir codigo paso paso',
 'herramienta comprobar programa

### embeddings

#### paraphrase-multilingual-MiniLM-L12-v2

In [50]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')   #Suitable Score Functions:	cosine-similarity (util.cos_sim)
global_data['embeddings']['mini_lm']['X_train'] = model.encode(global_data['x']['X_train'])
global_data['embeddings']['mini_lm']['X_test'] = model.encode(global_data['x']['X_test'])
global_data['embeddings']['mini_lm']['X_train']

array([[-0.18534315, -0.23511615, -0.25872898, ...,  0.01042269,
         0.3490764 , -0.02392072],
       [ 0.07902252, -0.03271351,  0.03022207, ...,  0.03590237,
         0.2753412 ,  0.04848982],
       [-0.08760416, -0.24450928, -0.10040322, ...,  0.326214  ,
         0.37647378, -0.06398531],
       ...,
       [-0.34795493, -0.02595298, -0.7502292 , ...,  0.10603917,
         0.03127527,  0.03923373],
       [-0.18579583,  0.06352917, -0.15932061, ...,  0.11662179,
         0.05348511, -0.14138183],
       [-0.17919302, -0.176184  , -0.29359305, ...,  0.2496631 ,
         0.06238774, -0.0445359 ]], dtype=float32)

In [51]:
# Normalize the embeddings to unit length
global_data['embeddings']['mini_lm']['X_train']= global_data['embeddings']['mini_lm']['X_train'] /  np.linalg.norm(global_data['embeddings']['mini_lm']['X_train'], axis=1, keepdims=True)
global_data['embeddings']['mini_lm']['X_test']= global_data['embeddings']['mini_lm']['X_test'] /  np.linalg.norm(global_data['embeddings']['mini_lm']['X_test'], axis=1, keepdims=True)
global_data['embeddings']['mini_lm']['X_train']

array([[-0.0531464 , -0.06741861, -0.07418949, ...,  0.00298866,
         0.10009625, -0.00685917],
       [ 0.02327453, -0.00963512,  0.00890132, ...,  0.01057434,
         0.08109635,  0.01428172],
       [-0.0215927 , -0.06026673, -0.02474742, ...,  0.08040534,
         0.09279339, -0.01577112],
       ...,
       [-0.08499248, -0.00633935, -0.18325314, ...,  0.02590143,
         0.00763939,  0.00958334],
       [-0.07069902,  0.02417412, -0.06062467, ...,  0.04437692,
         0.02035215, -0.05379861],
       [-0.05347627, -0.05257829, -0.08761647, ...,  0.07450653,
         0.01861827, -0.01329077]], dtype=float32)

#### paraphrase-multilingual-mpnet-base-v2

In [52]:

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')   #Suitable Score Functions:	cosine-similarity (util.cos_sim)
global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train'] = model.encode(global_data['x']['X_train'])
global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_test'] = model.encode(global_data['x']['X_test'])
global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train']

# Normalize the embeddings to unit length
global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train']= global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train'] /  np.linalg.norm(global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train'], axis=1, keepdims=True)
global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_test']= global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_test'] /  np.linalg.norm(global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_test'], axis=1, keepdims=True)
global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train']


array([[ 0.03575202,  0.0512767 , -0.00364097, ..., -0.01785721,
         0.0191138 , -0.01022901],
       [ 0.06204061,  0.07929561, -0.00668526, ...,  0.01621494,
         0.00904687, -0.00589262],
       [-0.04436562,  0.05408637, -0.00344007, ..., -0.01063866,
        -0.0130216 , -0.02835387],
       ...,
       [-0.03745706,  0.01732143, -0.00209869, ..., -0.0036179 ,
         0.03439554, -0.03138737],
       [-0.06358395,  0.00600331, -0.0073289 , ...,  0.02037926,
        -0.04084361, -0.05180229],
       [-0.04748544,  0.0154083 , -0.00500456, ..., -0.00294859,
        -0.00355369, -0.01019815]], dtype=float32)

#### paraphrase-MiniLM-L3-v2

In [53]:

model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')   #Suitable Score Functions:	cosine-similarity (util.cos_sim)
global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_train'] = model.encode(global_data['x']['X_train'])
global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_test'] = model.encode(global_data['x']['X_test'])
global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_train']

# Normalize the embeddings to unit length
global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_train']= global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_train'] /  np.linalg.norm(global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_train'], axis=1, keepdims=True)
global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_test']= global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_test'] /  np.linalg.norm(global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_test'], axis=1, keepdims=True)
global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_train']


array([[-0.07309177, -0.0321472 ,  0.00971462, ..., -0.03295293,
         0.11725953,  0.01546738],
       [-0.02348806, -0.02162573,  0.02659006, ..., -0.03151395,
         0.05050972, -0.04387247],
       [-0.14222948,  0.00036316,  0.00775755, ...,  0.06364495,
         0.10518659,  0.01724301],
       ...,
       [-0.0703008 ,  0.00629327, -0.04758104, ...,  0.01505081,
         0.01471139,  0.02739392],
       [-0.05332605,  0.0234894 , -0.02797176, ..., -0.03529615,
         0.13710056, -0.00357718],
       [-0.04259031, -0.00578414, -0.06187751, ...,  0.0451532 ,
         0.01657736,  0.0846616 ]], dtype=float32)

#### all-MiniLM-L12-v2

In [54]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')   #Suitable Score Functions:	cosine-similarity (util.cos_sim)
global_data['embeddings']['all-MiniLM-L12-v2']['X_train'] = model.encode(global_data['x']['X_train'])
global_data['embeddings']['all-MiniLM-L12-v2']['X_test'] = model.encode(global_data['x']['X_test'])
global_data['embeddings']['all-MiniLM-L12-v2']['X_train']

# Normalize the embeddings to unit length
global_data['embeddings']['all-MiniLM-L12-v2']['X_train']= global_data['embeddings']['all-MiniLM-L12-v2']['X_train'] /  np.linalg.norm(global_data['embeddings']['all-MiniLM-L12-v2']['X_train'], axis=1, keepdims=True)
global_data['embeddings']['all-MiniLM-L12-v2']['X_test']= global_data['embeddings']['all-MiniLM-L12-v2']['X_test'] /  np.linalg.norm(global_data['embeddings']['all-MiniLM-L12-v2']['X_test'], axis=1, keepdims=True)
global_data['embeddings']['all-MiniLM-L12-v2']['X_train']


array([[-0.08990403, -0.05086814, -0.04381368, ...,  0.01099263,
         0.09727875, -0.05833861],
       [-0.09543442, -0.01348669,  0.03267203, ..., -0.00764876,
         0.07163721, -0.05469358],
       [-0.0254356 , -0.06103364, -0.06905188, ...,  0.04308814,
         0.04850314, -0.08196529],
       ...,
       [-0.05479469,  0.04755693, -0.063356  , ...,  0.05041936,
         0.04835566, -0.11080009],
       [-0.00738352,  0.05673702,  0.0009508 , ...,  0.02748832,
         0.08716841,  0.01888164],
       [-0.0632618 ,  0.0033541 , -0.02743781, ...,  0.06101838,
        -0.02561231, -0.0592609 ]], dtype=float32)

#### all-MiniLM-L6-v2

In [55]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')   #Suitable Score Functions:	cosine-similarity (util.cos_sim)
global_data['embeddings']['all-MiniLM-L6-v2']['X_train'] = model.encode(global_data['x']['X_train'])
global_data['embeddings']['all-MiniLM-L6-v2']['X_test'] = model.encode(global_data['x']['X_test'])
global_data['embeddings']['all-MiniLM-L6-v2']['X_train']

# Normalize the embeddings to unit length
global_data['embeddings']['all-MiniLM-L6-v2']['X_train']= global_data['embeddings']['all-MiniLM-L6-v2']['X_train'] /  np.linalg.norm(global_data['embeddings']['all-MiniLM-L6-v2']['X_train'], axis=1, keepdims=True)
global_data['embeddings']['all-MiniLM-L6-v2']['X_test']= global_data['embeddings']['all-MiniLM-L6-v2']['X_test'] /  np.linalg.norm(global_data['embeddings']['all-MiniLM-L6-v2']['X_test'], axis=1, keepdims=True)
global_data['embeddings']['all-MiniLM-L6-v2']['X_train']


array([[ 0.00596678,  0.00309125, -0.07548715, ...,  0.07960682,
         0.02487338, -0.05006995],
       [ 0.0135451 ,  0.05599565, -0.02944525, ...,  0.05118658,
         0.01014403, -0.07628465],
       [-0.04660696,  0.02026807, -0.11785863, ...,  0.07585675,
         0.10371792, -0.05058546],
       ...,
       [-0.0738754 ,  0.0458301 , -0.05266656, ...,  0.11086688,
         0.03570692, -0.06907459],
       [ 0.01922256,  0.05540026, -0.03223083, ...,  0.10458876,
         0.05639042, -0.00223403],
       [-0.04188528,  0.00900187, -0.06700356, ...,  0.09139895,
        -0.02338452, -0.03455703]], dtype=float32)

#### USE (universal sentence encoder) like SBERT

In [56]:
## google embeddings model  https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3
#!pip install --upgrade tensorflow-estimator==2.3.0
#!pip install --upgrade tensorflow-hub
tf.compat.v1.estimator.experimental.dnn_logit_fn_builder
import tensorflow_hub as hub
import tensorflow_text  # this needs to be imported to set up some stuff in the background
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
global_data['embeddings']['use']['X_train']=embed(global_data['x']['X_train'])
global_data['embeddings']['use']['X_test']=embed(global_data['x']['X_test'])
global_data['embeddings']['use']['X_train'] #The input to the module is variable length text in any of the aforementioned languages and the output is a 512 dimensional vector

<tf.Tensor: shape=(343, 512), dtype=float32, numpy=
array([[ 0.02585286, -0.01451968,  0.07117936, ...,  0.04716099,
         0.01817636,  0.04881041],
       [ 0.06852595,  0.02283849,  0.04610641, ..., -0.07405574,
         0.0968125 ,  0.03021622],
       [-0.01862125, -0.0303612 ,  0.02471133, ..., -0.07862799,
        -0.07159283, -0.04643651],
       ...,
       [ 0.0187074 ,  0.0109222 , -0.01037703, ..., -0.02055839,
        -0.02540759,  0.03665491],
       [-0.05331954, -0.0244174 ,  0.02134675, ...,  0.01714571,
        -0.02872631, -0.00879866],
       [ 0.01787107,  0.02335536,  0.03486868, ..., -0.00266657,
        -0.0090678 ,  0.00509539]], dtype=float32)>

#### Glove
 It maps sentences & paragraphs to a 300 dimensional dense vector space and can be used for tasks like clustering or semantic search.

In [57]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/average_word_embeddings_glove.6B.300d')
global_data['embeddings']['glove']['X_train'] = model.encode(global_data['x']['X_train'])
global_data['embeddings']['glove']['X_test'] = model.encode(global_data['x']['X_test'])
global_data['embeddings']['glove']['X_train'].shape

(343, 300)

In [58]:
## add the same but with noramlized data and maybe stemmized

#### tf–idf short for Term Frequency Inverse Document Frequency.

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(max_features =1000)
tfid = tfidfVectorizer.fit(global_data['all_corpus']['comments'])
global_data['embeddings']['tfid']['X_train'] = tfid.transform(global_data['x']['X_train'])
global_data['embeddings']['tfid']['X_test'] = tfid.transform(global_data['x']['X_test'])
global_data['embeddings']['tfid']['X_train'].shape

(343, 1000)

In [60]:
# same but stemmized
tfidfVectorizer = TfidfVectorizer(max_features =1000)
tfid = tfidfVectorizer.fit(global_data['all_corpus']['comments_stemmized'])
global_data['embeddings']['tfid']['X_train_stemmized'] = tfid.transform(global_data['x']['X_train_stemmized']).toarray()
global_data['embeddings']['tfid']['X_test_stemmized'] = tfid.transform(global_data['x']['X_test_stemmized']).toarray()
global_data['embeddings']['tfid']['X_train_stemmized'].shape

(343, 717)

In [61]:
# same but lemmanized
tfidfVectorizer = TfidfVectorizer(max_features =1000)
tfid = tfidfVectorizer.fit(global_data['all_corpus']['comments_lemmanized'])
global_data['embeddings']['tfid']['X_train_lemmanized'] = tfid.transform(global_data['x']['X_train_lemmanized']).toarray()
global_data['embeddings']['tfid']['X_test_lemmanized'] = tfid.transform(global_data['x']['X_test_lemmanized']).toarray()
global_data['embeddings']['tfid']['X_train_lemmanized'].shape

(343, 1000)

#### combinations

In [63]:
global_data['embeddings']['concatenation_mini_l_384_dimmentions_and_tfid_stemmized']['X_train'] =  np.concatenate((global_data['embeddings']['mini_lm']['X_train'],global_data['embeddings']['tfid']['X_train_stemmized']), axis=1)

# unsupervised results

In [69]:
# iterate over the rows of the array
y_train_categories_string_dense = []
y_train_categories_string_dense = ['' for i in range(len(global_data['y']['y_train_categories']))]

counter = 0
for row in global_data['y']['y_train_categories']:
    # find the indices of the elements that equal 1 in the row
    indices = np.where(row != 0)[0]
    positions = ""
    # append the indices to the string
    for idx in indices:
        positions += f"{idx},"
    # remove the last comma from the string
    positions = positions.rstrip(",")
    y_train_categories_string_dense[counter] = positions
    counter = counter + 1

y_train_categories_string_dense

['5',
 '5',
 '1',
 '4,5',
 '1,3',
 '4',
 '1,4',
 '5',
 '4,5',
 '3,4',
 '1,3',
 '4',
 '1',
 '5',
 '3,4',
 '4',
 '2,5',
 '1,3',
 '4',
 '3,4',
 '4',
 '1,3,4',
 '1',
 '1,4',
 '4',
 '5',
 '1,4',
 '4',
 '1,4',
 '5',
 '1,4',
 '1,3,5',
 '1',
 '5',
 '4',
 '1',
 '4',
 '4',
 '1,3',
 '1',
 '1,3',
 '1,3,4',
 '1,3,4',
 '3,4',
 '3',
 '1,4',
 '1,3,4',
 '0,1,3',
 '1,3',
 '4',
 '4',
 '4',
 '1,3,5',
 '1',
 '4',
 '3',
 '1',
 '4',
 '3',
 '5',
 '4',
 '1,3',
 '1',
 '1',
 '1,4',
 '1,3,4',
 '1,3',
 '5',
 '3,4',
 '1,3',
 '4',
 '1,3,4',
 '1,3',
 '2',
 '4',
 '4,5',
 '1,3',
 '1,4',
 '1,4',
 '1,3',
 '1,3,4',
 '5',
 '3,4',
 '5',
 '1,3',
 '4',
 '1,3',
 '1,4',
 '5',
 '0,1,4',
 '1,3,4',
 '1,3,5',
 '1,3',
 '4',
 '5',
 '3',
 '1',
 '5',
 '4',
 '4',
 '3,4',
 '4',
 '1,3',
 '1,4',
 '1,4',
 '1,3',
 '4',
 '2',
 '2,5',
 '3,4',
 '0,5',
 '5',
 '4',
 '4',
 '3',
 '1,5',
 '4',
 '1,3,4',
 '3,4',
 '0,2,5',
 '1,3',
 '1,4,5',
 '4',
 '4',
 '4',
 '5',
 '1,4',
 '4',
 '4,5',
 '5',
 '1,4',
 '5',
 '1',
 '1,4',
 '5',
 '4',
 '4',
 '1,4',
 '4',


In [70]:
global_data['y']['y_train_categories']

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 1]])

In [71]:
# iterate over the rows of the array
y_train_sub_categories_string_dense = []
y_train_sub_categories_string_dense = ['' for i in range(len(global_data['y']['y_train_sub_categories']))]

counter = 0
for row in global_data['y']['y_train_sub_categories']:
    # find the indices of the elements that equal 1 in the row
    indices = np.where(row != 0)[0]
    positions = ""
    # append the indices to the string
    for idx in indices:
        positions += f"{idx},"
    # remove the last comma from the string
    positions = positions.rstrip(",")
    y_train_sub_categories_string_dense[counter] = positions
    counter = counter + 1

y_train_sub_categories_string_dense

['18',
 '18',
 '3,5',
 '14,18',
 '1,8',
 '15',
 '3,5,16',
 '17',
 '15,17',
 '10,12,13',
 '1,10',
 '16',
 '5',
 '18',
 '10,16',
 '11',
 '18',
 '5,8',
 '16',
 '10,15',
 '12,13',
 '3,12,13',
 '5',
 '1,14,16',
 '11',
 '17',
 '1,2,16',
 '16',
 '7,12,13',
 '17,18',
 '7,12,13',
 '1,9,10,20',
 '0',
 '20',
 '14',
 '1',
 '14',
 '14,15,16',
 '2,4,10',
 '5',
 '10',
 '0,1,10,12,13',
 '1,2,9,12,13',
 '8,10,16',
 '10',
 '1,11,15,16',
 '1,5,14',
 '1',
 '4,8',
 '14',
 '14',
 '14',
 '1,8,10,17',
 '5',
 '14,15',
 '10',
 '1',
 '14',
 '8',
 '17',
 '14',
 '1,5,9',
 '1',
 '5',
 '1,12,13,16',
 '5,10,14,16',
 '5,9',
 '19,20',
 '10,16',
 '1,2,8',
 '15,16',
 '1,9,11',
 '5,9',
 '',
 '15',
 '14,19',
 '1,8,10',
 '1,12,13,14',
 '7,14',
 '1,10',
 '5,11',
 '17',
 '10,16',
 '17',
 '1,9,10',
 '12,13',
 '0,10',
 '5,14',
 '18',
 '1,14',
 '0,1,9,14',
 '1,9,10,20',
 '5,10',
 '12,13',
 '17',
 '10',
 '1',
 '17',
 '12,13',
 '11,12,13',
 '10,14,15',
 '12,13,15',
 '1,3,8,10',
 '1,5,14',
 '2,7,11,12,13,15',
 '1,10',
 '16',
 '',
 

In [72]:
comments_df= pd.DataFrame()
comments_df['data_index'] = global_data['x']['X_train']
comments_df['real_categories'] = y_train_categories_string_dense
comments_df['real_sub_categories'] = y_train_sub_categories_string_dense

comments_df

,data_index,real_categories,real_sub_categories
0,"es un buen programa, lo unico es que es algo c...",5,18
1,"no lo usamos muchas veces, y al usarlo, nos ob...",5,18
2,permitio la creacion de varios programas en un...,1,"3,5"
3,"es una buena forma de calificar, aunque hay qu...","4,5","14,18"
4,es muy util el hecho de que pruebe casos y que...,"1,3","1,8"
...,...,...,...
338,es una gran herramienta que da un excelente fe...,"1,3","2,10"
339,la opcion en la que uno puede ver que esta hac...,"1,3","2,10"
340,uncode es una herramienta muy util debido a qu...,"1,3","2,10"
341,"facilito el envio de talleres, ademas corrige ...",4,14


In [73]:
visualization_df= pd.DataFrame()

def most_similar(doc_id,similarity_matrix,matrix, df):
  visualization_df = pd.DataFrame(columns=df.columns)
  if matrix=='Cosine Similarity':
      similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
  for ix in similar_ix:
      if ix==doc_id:
          continue
      else:
          row = df.iloc[ix].copy()
          row['Similarity'] = similarity_matrix[doc_id][ix]
          visualization_df = pd.concat([visualization_df, row.to_frame().T])

  return visualization_df

id_doc_seleccionado = 3

id_doc_seleccionado2 = 5

id_doc_seleccionado3 = 10

In [66]:
def euclidean_distances(pairwise_similarities, temp_df):
  averageEuclideanDistanceTotal = 0
  euclideanDistanceTotal = 0
  averageEuclideanDistanceTotal_subCategories = 0
  euclideanDistanceTotal_subCategories = 0
  numberOfComments = 30
  for i in range(0,len(pairwise_similarities)):
    res = most_similar(i,pairwise_similarities,'Cosine Similarity', temp_df)
    mostSimilarIndex = res[0:numberOfComments].index
    totalDistance = 0
    averageEuclideanDistance = 0
    totalDistance_subCategories = 0
    averageEuclideanDistance_subCategories = 0
    for index in mostSimilarIndex:
      baseCategorie = global_data['y']['y_train_categories'][i]
      categorie = global_data['y']['y_train_categories'][index]
      baseCategorie_subCategories = global_data['y']['y_train_sub_categories'][i]
      categorie_subCategories = global_data['y']['y_train_sub_categories'][index]
      euclideanDistance = np.linalg.norm(categorie - baseCategorie)
      totalDistance = totalDistance + euclideanDistance
      euclideanDistance_subCategories = np.linalg.norm(categorie_subCategories - baseCategorie_subCategories)
      totalDistance_subCategories = totalDistance_subCategories + euclideanDistance_subCategories
    averageEuclideanDistance = totalDistance/numberOfComments
    euclideanDistanceTotal = euclideanDistanceTotal + averageEuclideanDistance
    averageEuclideanDistance_subCategories = totalDistance_subCategories/numberOfComments
    euclideanDistanceTotal_subCategories = euclideanDistanceTotal_subCategories + averageEuclideanDistance_subCategories
  averageEuclideanDistanceTotal = euclideanDistanceTotal/len(pairwise_similarities)
  averageEuclideanDistanceTotal_subCategories = euclideanDistanceTotal_subCategories/len(pairwise_similarities)
  return averageEuclideanDistanceTotal , averageEuclideanDistanceTotal_subCategories




In [74]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['use']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.143676090186919, 1.6902269343217364)

In [75]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['concatenation_mini_l_384_dimmentions_and_tfid_stemmized']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.0723793787364848, 1.6186865807025923)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['tfid']['X_train_stemmized'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.1321780800773276, 1.6474327114377223)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['tfid']['X_train_lemmanized'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.146466526797997, 1.6566001224374323)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['glove']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.248998911485625, 1.88888464157062)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['mini_lm']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.0721300973900552, 1.633477397567147)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['all-MiniLM-L6-v2']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.1771473097419412, 1.7026630115218684)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['all-MiniLM-L12-v2']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.1710840163155802, 1.666376336639678)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.0741991276258447, 1.619073145509825)

In [ ]:
pairwise_similarities=cosine_similarity(global_data['embeddings']['paraphrase-MiniLM-L3-v2']['X_train'])

temp_df = comments_df.copy()
res = euclidean_distances(pairwise_similarities, temp_df)
res

(1.1956686091941482, 1.78317573625131)

### similarities in table form

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['paraphrase-multilingual-mpnet-base-v2']['X_train'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
res = most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)
res[0:20]

es una buena forma de calificar, aunque hay que ser demasiado especifico en los resultados
data_index             es una buena forma de calificar, aunque hay qu...
real_categories                                                      4,5
real_sub_categories                                                14,18
Name: 3, dtype: object


,data_index,real_categories,real_sub_categories,Similarity
180,"es una exelente herramienta de evaluacion, y p...","1,4","7,16",0.8182
241,solo es una herramienta para calificar.,4,14,0.781879
123,es una plataforma que es muy util para calificar,4,14,0.711281
281,en cierta medida ayuda a saber mi rendimiento ...,"1,4","2,16",0.675294
167,es una plataforma bien interesante donde de ve...,4,16,0.672719
90,porque es una plataforma interesante que adema...,"1,3,4","0,1,9,14",0.667249
184,es muy conveniente el poder recibir una califi...,"1,3,4","1,9,14",0.656121
266,es una buena herramienta para calificar los tr...,4,14,0.63872
149,opino que es bastante util ya que te hace en e...,"1,3,4","1,8,14,16",0.634772
143,"en segundo lugar, la plataforma califica con r...","1,3,4","1,2,8,10,14",0.619022


In [ ]:
global_data['y']['y_train_categories']

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 1]])

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

porque permite aplicar los conocimientos ensenados sobre programacion de forma similar a la que se haria en un computador por medio de python.
data_index             porque permite aplicar los conocimientos ensen...
real_categories                                                        4
real_sub_categories                                                   15
Name: 5, dtype: object


,data_index,real_categories,real_sub_categories,Similarity
294,por que facilita al estudiante tener un lugar ...,"1,3,4","4,10,16",0.731353
256,porque presenta herramientas utiles que permit...,"1,3",10,0.725462
69,"porque permite probar el codigo, hacer el segu...","1,3","1,2,8",0.682965
100,porque cuenta con herramientas bastante utiles...,"3,4","10,14,15",0.68009
270,debido a que es una herramienta que proporcion...,4,15,0.655157
...,...,...,...,...
265,falla a veces,5,17,0.036492
32,los casos de prueba ayudaron bastante,1,0,0.028388
172,el juez es cruel pero justo,4,14,0.010576
81,permanece caido,5,17,-0.001983


In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

fue util, para verificar nuestros fallos y seguir el codigo paso a paso
data_index             fue util, para verificar nuestros fallos y seg...
real_categories                                                      1,3
real_sub_categories                                                 1,10
Name: 10, dtype: object


,data_index,real_categories,real_sub_categories,Similarity
195,fue util y practica a la hora de probar mi codigo,"1,3,4","5,8,16",0.792141
181,"ha sido util para subir ejercicios, comprobarl...","1,3,4","1,9,14,16",0.718895
152,fue util porque me enseno estrictamente a mane...,"1,4","2,15",0.691148
247,fue util para realizar los talleres designados...,"1,3,4","1,5,9,10,14",0.682009
225,fue util porque al poder obtener una retroalim...,"1,4","2,14,16",0.663964
...,...,...,...,...
119,existen plataformas con un ambiente un poco ma...,"0,2,5",17,0.075291
278,provee un entorno amplio de retroalimentacion ...,1,2,0.070944
326,"los talleres dejados por el profesor, refuerza...",4,"12,13",0.058653
81,permanece caido,5,17,0.050749


##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['use']['X_train_categorical_calification'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

KeyError: ignored

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['use']['X_train_one_hot_calification'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['mini_lm']['X_train'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['mini_lm']['X_train_categorical_calification'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['mini_lm']['X_train_one_hot_calification'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['glove']['X_train'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['glove']['X_train_categorical_calification'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)

##### similarity  dot product with USE embedding


In [ ]:

pairwise_similarities=cosine_similarity(global_data['embeddings']['tfid']['X_train_stemmized'])

print(comments_df.iloc[id_doc_seleccionado]['data_index'])
print(comments_df.iloc[id_doc_seleccionado])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado2]['data_index'])
print(comments_df.iloc[id_doc_seleccionado2])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado2,pairwise_similarities,'Cosine Similarity', temp_df)

In [ ]:
print(comments_df.iloc[id_doc_seleccionado3]['data_index'])
print(comments_df.iloc[id_doc_seleccionado3])

temp_df = comments_df.copy()
most_similar(id_doc_seleccionado3,pairwise_similarities,'Cosine Similarity', temp_df)